# Prepare OC excel from CSV files
make csv file using parse.py

## Autocad POSTPROCESSING

In [165]:
import pandas as pd
from pandas import ExcelWriter
from openpyxl import load_workbook
import numpy as np
import csv
from rdp import rdp
from tqdm import tqdm

In [166]:
# df = pd.read_csv("MOM-K-56-PL-DW-5003-01 REV Z0_Overall Field Layout_ASB -  for oc_1e56c9f24.logout.csv")
# fname="Kikeh Field 2014_Ref dw Timb48_database_18_Dec_2013 _[BA24.05.17BOTTOM HOLE LOCATION BASE H120] for OC_1f72b41c6.logout.csv"
fname = "Kikeh Phase3 - Layout SMA_12f8edf86.logout.csv"
df = pd.read_csv(fname)
df

,Handle,Layer Name,Layer Type,Block name,Block rotation,No,Wtf,X,Y,Z
0,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,1,at,198807.18,649731.71,0.0
1,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,2,at,198804.91,649732.77,0.0
2,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,3,at,198802.77,649733.47,0.0
3,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,4,at,198801.80,649733.57,0.0
4,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,5,at,198800.56,649733.31,0.0
...,...,...,...,...,...,...,...,...,...,...
2613,1de22,equi,BLOCK REFERENCE,A$C32066C52,30.0,1,at,200960.04,646702.99,0.0
2614,1de21,equi,BLOCK REFERENCE,A$C32066C52,85.0,1,at,200972.05,646592.15,0.0
2615,1de20,equi,BLOCK REFERENCE,A$C32066C52,90.0,1,at,200832.51,646650.62,0.0
2616,1de1f,equi,BLOCK REFERENCE,A$C32066C52,265.0,1,at,199188.53,646848.33,0.0


In [167]:
# define the project location and substract the values from XY
# projectLocation = (df[df["Block name"]== 'fpso'].X.values[0],df[df["Block name"]== 'fpso'].Y.values[0])

# in case that FPSO is missing - add manual position
projectLocation = (200324.0024, 650270.9825)
print (projectLocation)
df.X=round(df.X-projectLocation[0],2)
df.Y=round(df.Y-projectLocation[1],2)
df

(200324.0024, 650270.9825)


,Handle,Layer Name,Layer Type,Block name,Block rotation,No,Wtf,X,Y,Z
0,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,1,at,-1516.82,-539.27,0.0
1,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,2,at,-1519.09,-538.21,0.0
2,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,3,at,-1521.23,-537.51,0.0
3,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,4,at,-1522.20,-537.41,0.0
4,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,5,at,-1523.44,-537.67,0.0
...,...,...,...,...,...,...,...,...,...,...
2613,1de22,equi,BLOCK REFERENCE,A$C32066C52,30.0,1,at,636.04,-3567.99,0.0
2614,1de21,equi,BLOCK REFERENCE,A$C32066C52,85.0,1,at,648.05,-3678.83,0.0
2615,1de20,equi,BLOCK REFERENCE,A$C32066C52,90.0,1,at,508.51,-3620.36,0.0
2616,1de1f,equi,BLOCK REFERENCE,A$C32066C52,265.0,1,at,-1135.47,-3422.65,0.0


In [168]:
# the same project location should be applied to the seabed
inputfile = "kikeh-seabed-utm-250m.csv"
subx = projectLocation[0]
suby = projectLocation[1]
outputfile = str(inputfile)+'_origin_'+str(subx)+'_'+str(suby)+'.csv'
with open(inputfile, 'r') as f_input: 
    reader = csv.reader(f_input, delimiter=',')
    with open(outputfile, 'w') as result:
        writer = csv.writer(result, delimiter=',')
        for row in reader:
            writer.writerow([float(row[0])-subx,float(row[1])-suby,float(row[2])])
print(f"Seabed with file name {outputfile} has been saved")
projectLocation


Seabed with file name kikeh-seabed-utm-250m.csv_origin_200324.0024_650270.9825.csv has been saved


(200324.0024, 650270.9825)

In [169]:
df.dtypes, df["Layer Type"].unique()

(Handle             object
 Layer Name         object
 Layer Type         object
 Block name         object
 Block rotation    float64
 No                  int64
 Wtf                object
 X                 float64
 Y                 float64
 Z                 float64
 dtype: object, array(['LWPOLYLINE', 'BLOCK REFERENCE'], dtype=object))

In [170]:
# remove concequtive points at the same location
_df = df[df.duplicated(subset=df.columns.difference(['No']),keep=False)]
print(f"Found {len(_df)} duplicates. Finding if they are concequtive and if so deleting them.")
i=-5
indexToDrop=[]
for index, row in _df.iterrows():
    if i==row.No-1:
        indexToDrop.append(index)
#         df.drop(df.index[[index]], inplace=True)
#         print(index, row)
    i=row.No
df.drop(df.index[indexToDrop], inplace=True)
df

Found 3 duplicates. Finding if they are concequtive and if so deleting them.


,Handle,Layer Name,Layer Type,Block name,Block rotation,No,Wtf,X,Y,Z
0,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,1,at,-1516.82,-539.27,0.0
1,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,2,at,-1519.09,-538.21,0.0
2,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,3,at,-1521.23,-537.51,0.0
3,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,4,at,-1522.20,-537.41,0.0
4,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,5,at,-1523.44,-537.67,0.0
...,...,...,...,...,...,...,...,...,...,...
2613,1de22,equi,BLOCK REFERENCE,A$C32066C52,30.0,1,at,636.04,-3567.99,0.0
2614,1de21,equi,BLOCK REFERENCE,A$C32066C52,85.0,1,at,648.05,-3678.83,0.0
2615,1de20,equi,BLOCK REFERENCE,A$C32066C52,90.0,1,at,508.51,-3620.36,0.0
2616,1de1f,equi,BLOCK REFERENCE,A$C32066C52,265.0,1,at,-1135.47,-3422.65,0.0


In [184]:
# first iteration showed that these layes are not valid so removing them
df = df.drop(df[(df["Layer Name"] == "equi") | 
                (df["Layer Name"] == "0") |
                (df['Layer Name'].str.contains(pat = 'Exist.')) | 
                (df['Layer Name'].str.contains(pat = 'WELD')) |
                (df['Block name'].str.contains(pat = ' ORI'))
               ].index)
df

,Handle,Layer Name,Layer Type,Block name,Block rotation,No,Wtf,X,Y,Z
0,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,1,at,-1516.82,-539.27,0.0
1,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,2,at,-1519.09,-538.21,0.0
2,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,3,at,-1521.23,-537.51,0.0
3,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,4,at,-1522.20,-537.41,0.0
4,37667,HFL PM-04,LWPOLYLINE,NaN,NaN,5,at,-1523.44,-537.67,0.0
...,...,...,...,...,...,...,...,...,...,...
2562,2fbff,UMBILCAL,LWPOLYLINE,NaN,NaN,8,at,-1694.50,-4729.10,0.0
2563,2fbff,UMBILCAL,LWPOLYLINE,NaN,NaN,9,at,-2128.69,-3981.15,0.0
2564,2fbff,UMBILCAL,LWPOLYLINE,NaN,NaN,10,at,-1558.42,-2982.56,0.0
2582,1de41,EXISTING TIE-IN,BLOCK REFERENCE,A$C26AB678C,45.0,1,at,-1809.97,-4058.80,0.0


In [185]:
def pointsToExcel(rdp_points):
    #OC format of points in excel
    points=''
    for index, i in enumerate(rdp_points):
        i = [ round(elem, 2) for elem in i ]
        if index==len(rdp_points)-1:
            points = points+str(i[0])+','+str(i[1])+','+str(i[2])
        else:
            points = points+str(i[0])+','+str(i[1])+','+str(i[2])+';'
    return points

In [186]:
# we have now list of points for different AUTOCAD types. Now we need to put one list for lines and one for structures. 
lt_uni=df["Layer Type"].unique() 
lout=[]
bout=[]
for lt in tqdm(lt_uni):
    print(f"Generating {lt} list of points and name")
    _df=df[df["Layer Type"]==lt]
    if lt=="LWPOLYLINE" :
        h_uni=_df["Handle"].unique()
        for h in (h_uni):
            _line = _df[_df["Handle"]==h]
            if len(_line)>1:
                points = rdp(_line[["X","Y","Z"]], epsilon=0.5)
                points = pointsToExcel(points)
                lname=_line["Layer Name"].iloc[0]
    #             print(f"processed {lname}")
                lout.append([lname,points])
    elif lt=="VERTEX":
        h_uni=_df["Layer Name"].unique()
        for h in h_uni:
            _line = _df[_df["Layer Name"]==h]
#             if len(_line)>5:
            points = rdp(_line[["X","Y","Z"]], epsilon=0.5)
            points = pointsToExcel(points)
            lname=_line["Layer Name"].iloc[0]
    #             print(f"processed {lname}")
            lout.append([lname,points])
#             print(out)
    elif lt=="BLOCK REFERENCE":
        h_uni=_df["Handle"].unique()
        for h in h_uni:
            _block=_df[_df["Handle"]==h]
            point = ','.join([str(_block.X.values[0]),str(_block.Y.values[0]),str(_block.Z.values[0])])
            bname = (_block["Layer Name"].values[0])
            bout.append([bname, _block["Block name"].values[0],_block["Block rotation"].values[0],point])
            


  0%|          | 0/2 [00:00<?, ?it/s]

Generating LWPOLYLINE list of points and name



100%|██████████| 2/2 [00:02<00:00,  1.12s/it]

Generating BLOCK REFERENCE list of points and name


In [187]:
# now we need to add columns needed for the OC for Lines. 
l_df = pd.DataFrame(lout)
l_df.columns=["Name", 'Points (x,y,z) (m)']
l_df.sort_values("Name",ascending=True, inplace=True)
l_df.reset_index(inplace=True)
l_df.drop(columns=['index'], inplace=True)
# l_df.Name.unique()
ll_df=pd.DataFrame()
# ll_df.columns=["Name", 'Points (x,y,z) (m)']
for l in tqdm(l_df.Name.unique()):
    _df=l_df[l_df["Name"]==l]
    if len(_df)>1:
        i=1
        for index, row in _df.iterrows():
            row.Name=row.Name+' '+ str(i)
            i+=1
    ll_df=pd.concat([ll_df, _df])
ll_df.index=ll_df.index+5000
ll_df["ParentId"]=""
ll_df["Asset type"]=""
ll_df["Status"]="Existing"
# ll_df["Status"]="Planned"
ll_df["Template name"]="flexible line"
ll_df.loc[ll_df['Name'].str.contains(pat = 'UMBILICAL'), 'Template name'] = 'umbilical line'
ll_df.loc[ll_df['Name'].str.contains(pat = 'UMBILCAL'), 'Template name'] = 'umbilical line'
ll_df.loc[ll_df['Name'].str.contains(pat = 'EFL'), 'Template name'] = 'EFL'
ll_df.loc[ll_df['Name'].str.contains(pat = 'HFL'), 'Template name'] = 'HFL'
ll_df=ll_df[["ParentId", "Name", "Asset type", "Template name", "Status", "Points (x,y,z) (m)"]]
ll_df


100%|██████████| 10/10 [00:00<00:00, 216.20it/s]


,ParentId,Name,Asset type,Template name,Status,"Points (x,y,z) (m)"
5000,,EFL PM-03 1,,EFL,Existing,"-1452.37,-3001.26,0.0;-1451.33,-3008.84,0.0;-1..."
5001,,EFL PM-03 2,,EFL,Existing,"-1467.42,-2966.76,0.0;-1472.84,-2961.9,0.0;-14..."
5002,,EFL PM-03 3,,EFL,Existing,"-1514.82,-2956.43,0.0;-1515.01,-2949.5,0.0;-15..."
5003,,EFL PM-03 4,,EFL,Existing,"-1512.64,-2957.36,0.0;-1506.81,-2957.43,0.0;-1..."
5004,,EFL PM-03 5,,EFL,Existing,"-1546.44,-2920.62,0.0;-1529.94,-2920.56,0.0;-1..."
5005,,EFL PM-03 6,,EFL,Existing,"-1512.64,-2957.48,0.0;-1498.7,-2958.23,0.0;-14..."
5006,,EFL PM-03 7,,EFL,Existing,"-1515.28,-2957.48,0.0;-1519.38,-2957.87,0.0;-1..."
5007,,EFL PM-03 8,,EFL,Existing,"-1515.28,-2957.74,0.0;-1519.45,-2958.21,0.0;-1..."
5008,,EFL PM-04 1,,EFL,Existing,"-1498.51,-583.24,0.0;-1499.27,-575.7,0.0;-1502..."
5009,,EFL PM-04 2,,EFL,Existing,"-1496.33,-584.29,0.0;-1488.42,-584.3,0.0;-1475..."


In [188]:
ll_df[ll_df["Template name"]=="flexible line"]

,ParentId,Name,Asset type,Template name,Status,"Points (x,y,z) (m)"
5028,,OIL FLEXIBLE JUMPER 1,,flexible line,Existing,"-1462.31,-2968.56,0.0;-1436.42,-2963.21,0.0"
5029,,OIL FLEXIBLE JUMPER 2,,flexible line,Existing,"-1462.31,-2970.76,0.0;-1436.96,-2986.79,0.0"
5030,,OIL FLEXIBLE JUMPER 3,,flexible line,Existing,"-1840.01,-4076.54,0.0;-1864.02,-4083.49,0.0"
5031,,OIL FLEXIBLE JUMPER 4,,flexible line,Existing,"-1468.59,-2968.56,0.0;-1490.53,-2948.1,0.0"
5032,,OIL FLEXIBLE JUMPER 5,,flexible line,Existing,"-1469.08,-2970.7,0.0;-1498.45,-2973.62,0.0"
5033,,OIL FLEXIBLE JUMPER 6,,flexible line,Existing,"-1828.85,-4040.27,0.0;-1856.27,-4040.27,0.0;-1..."
5034,,OIL FLEXIBLE JUMPER 7,,flexible line,Existing,"-1828.85,-4042.36,0.0;-1856.25,-4043.42,0.0;-1..."
5035,,OIL FLEXIBLE JUMPER 8,,flexible line,Existing,"-1837.28,-4072.14,0.0;-1825.73,-4044.45,0.0"
5036,,OIL FLEXIBLE JUMPER 9,,flexible line,Existing,"-1837.94,-4077.27,0.0;-1831.29,-4104.64,0.0"
5037,,OIL FLEXIBLE JUMPER 10,,flexible line,Existing,"-1462.31,-2972.96,0.0;-1452.37,-3001.26,0.0"


In [189]:
# checking if there is for some line too many items. 
l_df["Name"].value_counts()

OIL FLEXIBLE JUMPER    17
EFL PM-03               8
EFL PM-04               7
HFL PM-04               7
UMBILCAL                3
HFL WM-04               3
EFL WM-04               3
OIL PRODUCTION          2
WI FLEXIBLE JUMPER      2
WATER INJECTION         2
Name: Name, dtype: int64

In [190]:
# checking for a strange number of elements repeating
l_df[l_df["Name"]=='0']

,Name,"Points (x,y,z) (m)"


In [191]:
# now we need to add columns needed for the OC for Structures. 
b_df = pd.DataFrame(bout)

b_df.columns=["Layer", "Block", "4 Azimuth (deg)", 'Position (x,y,z) (m)']
b_df["Name"] = b_df.Layer +" "+ b_df.Block
b_df.sort_values("Name",ascending=True, inplace=True)
b_df.reset_index(inplace=True)
b_df.drop(columns=['index'], inplace=True)
# b_df.Name.unique()
b_df["Template name"]="OC-box"
bb_df=pd.DataFrame()
# ll_df.columns=["Name", 'Points (x,y,z) (m)']
for b in tqdm(b_df.Name.unique()):
    _df=b_df[b_df["Name"]==b]
    if len(_df)>1:
        i=1
        for index, row in _df.iterrows():
            _df.loc[index,"Name"]=row.Name+' '+str(i)
            i+=1
#         print(_df)
    bb_df=pd.concat([bb_df, _df])
bb_df.index=bb_df.index+100
bb_df["ParentId"]=""
bb_df["Asset type"]=""
bb_df["Status"]="Existing"
# bb_df["Status"]="Planned"
bb_df.loc[bb_df['Name'].str.contains(pat = 'WM-'), 'Template name'] = 'OC-manifold'
bb_df.loc[bb_df['Name'].str.contains(pat = 'PM-'), 'Template name'] = 'OC-manifold'
bb_df.loc[bb_df['Name'].str.contains(pat = ' gm'), 'Template name'] = 'OC-manifold'
bb_df.loc[bb_df['Name'].str.contains(pat = 'manifold'), 'Template name'] = 'OC-manifold'
bb_df.loc[bb_df['Name'].str.contains(pat = 'Manifold'), 'Template name'] = 'OC-manifold'
bb_df.loc[bb_df['Name'].str.contains(pat = 'fpso'), 'Template name'] = 'OC-fpso'
bb_df.loc[bb_df['Name'].str.contains(pat = 'well'), 'Template name'] = 'OC-xmas'
bb_df.loc[bb_df['Name'].str.contains(pat = 'Well'), 'Template name'] = 'OC-xmas'
bb_df.loc[bb_df['Name'].str.contains(pat = ' xt'), 'Template name'] = 'OC-xmas'
bb_df.loc[bb_df['Name'].str.contains(pat = ' wx'), 'Template name'] = 'OC-xmas'
bb_df.loc[bb_df['Name'].str.contains(pat = 'xmt'), 'Template name'] = 'OC-xmas'
bb_df.loc[bb_df['Name'].str.contains(pat = 'XMT'), 'Template name'] = 'OC-xmas'
bb_df.loc[bb_df['Name'].str.contains(pat = 'plet'), 'Template name'] = 'OC-plet'
bb_df.loc[bb_df['Name'].str.contains(pat = 'ilt'), 'Template name'] = 'OC-ilt'
bb_df.loc[bb_df['Name'].str.contains(pat = 'sdu'), 'Template name'] = 'OC-sdu'
bb_df.loc[bb_df['Name'].str.contains(pat = 'SDU'), 'Template name'] = 'OC-sdu'
bb_df.loc[bb_df['Name'].str.contains(pat = 'UTA'), 'Template name'] = 'OC-uta'

bb_df=bb_df[["ParentId", "Name", "Asset type", "Template name", "Status", "Position (x,y,z) (m)", "4 Azimuth (deg)"]]
bb_df


100%|██████████| 29/29 [00:00<00:00, 233.88it/s]


,ParentId,Name,Asset type,Template name,Status,"Position (x,y,z) (m)",4 Azimuth (deg)
100,,Dim unknown structure 2,,OC-box,Existing,"-1757.0,-4060.98,0.0",0.0
101,,EXISTING TIE-IN A$C26AB678C,,OC-box,Existing,"-1809.97,-4058.8,0.0",45.0
102,,EXISTING TIE-IN PM-02,,OC-manifold,Existing,"-1824.63,-4041.31,0.0",0.0
103,,EXISTING TIE-IN S-RBP-ET3,,OC-box,Existing,"725.37,734.29,0.0",0.0
104,,KIKEH3 STRUCTURES A$C1115347E 1,,OC-box,Existing,"-1838.13,-4074.52,0.0",160.0
105,,KIKEH3 STRUCTURES A$C1115347E 2,,OC-box,Existing,"-1817.88,-4071.39,0.0",195.0
106,,KIKEH3 STRUCTURES PM-03,,OC-manifold,Existing,"-1465.45,-2970.76,0.0",0.0
107,,KIKEH3 STRUCTURES PM-04,,OC-manifold,Existing,"-1449.15,-597.58,0.0",272.0
108,,KIKEH3 STRUCTURES SDU (PM-03),,OC-sdu,Existing,"-1513.96,-2958.64,0.0",0.0
109,,KIKEH3 STRUCTURES SDU (PM-04),,OC-sdu,Existing,"-1497.65,-585.45,0.0",0.0


In [192]:
bb_df[bb_df["Template name"]=="OC-box"]

,ParentId,Name,Asset type,Template name,Status,"Position (x,y,z) (m)",4 Azimuth (deg)
100,,Dim unknown structure 2,,OC-box,Existing,"-1757.0,-4060.98,0.0",0.0
101,,EXISTING TIE-IN A$C26AB678C,,OC-box,Existing,"-1809.97,-4058.8,0.0",45.0
103,,EXISTING TIE-IN S-RBP-ET3,,OC-box,Existing,"725.37,734.29,0.0",0.0
104,,KIKEH3 STRUCTURES A$C1115347E 1,,OC-box,Existing,"-1838.13,-4074.52,0.0",160.0
105,,KIKEH3 STRUCTURES A$C1115347E 2,,OC-box,Existing,"-1817.88,-4071.39,0.0",195.0


In [193]:
# adjusting rotations 
# For this case only Autocad rotations adjustment TO BE CHECKED FOR THE NEXT
for index, b in list(bb_df.iterrows()):
    if b["Template name"]=="OC-ilt":
        bb_df.loc[index,"4 Azimuth (deg)"]=360-b["4 Azimuth (deg)"]+90
    else:
        bb_df.loc[index,"4 Azimuth (deg)"]=360-b["4 Azimuth (deg)"]
bb_df

,ParentId,Name,Asset type,Template name,Status,"Position (x,y,z) (m)",4 Azimuth (deg)
100,,Dim unknown structure 2,,OC-box,Existing,"-1757.0,-4060.98,0.0",360.0
101,,EXISTING TIE-IN A$C26AB678C,,OC-box,Existing,"-1809.97,-4058.8,0.0",315.0
102,,EXISTING TIE-IN PM-02,,OC-manifold,Existing,"-1824.63,-4041.31,0.0",360.0
103,,EXISTING TIE-IN S-RBP-ET3,,OC-box,Existing,"725.37,734.29,0.0",360.0
104,,KIKEH3 STRUCTURES A$C1115347E 1,,OC-box,Existing,"-1838.13,-4074.52,0.0",200.0
105,,KIKEH3 STRUCTURES A$C1115347E 2,,OC-box,Existing,"-1817.88,-4071.39,0.0",165.0
106,,KIKEH3 STRUCTURES PM-03,,OC-manifold,Existing,"-1465.45,-2970.76,0.0",360.0
107,,KIKEH3 STRUCTURES PM-04,,OC-manifold,Existing,"-1449.15,-597.58,0.0",88.0
108,,KIKEH3 STRUCTURES SDU (PM-03),,OC-sdu,Existing,"-1513.96,-2958.64,0.0",360.0
109,,KIKEH3 STRUCTURES SDU (PM-04),,OC-sdu,Existing,"-1497.65,-585.45,0.0",360.0


In [194]:

def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()

In [195]:
# save excel with 2 sheets 
fname="kikeh3_fromACAD_rev1.xlsx"
save_xls([ll_df,bb_df], fname)

In [196]:
# now we modify excel to be in OC format for direct import
structures_data = [
    ["Property Name",'','','',"templateName","status","position","azimuth"],
    ["Property Type Value",'','','',"string","string","vector","number"],
    ["Property Type",'','','',"Engineering","Engineering","Engineering","Engineering"],
    ["Unit",'','','','','','','','','']
]
lines_data = [
    ["Property Name",'','','',"templateName","status",'points'],
    ["Property Type Value",'','','',"string","string","vectorArray"],
    ["Property Type",'','','',"Engineering","Engineering","Engineering"],
    ["Unit",'','','','','','','']
]

wb = load_workbook(fname)
ws = wb.worksheets[0]
ws.insert_rows(1,amount=4)
ws.title="lines"
for ri,row_data in enumerate(lines_data):
    for ci, col_data in enumerate(row_data):
        ws.cell(row=ri+1,column=ci+1,value=col_data)

ws = wb.worksheets[1]
ws.insert_rows(1,amount=4)
ws.title="structures"
# Append 2 new Rows - Columns A - D
for ri, row_data in enumerate(structures_data):
    for ci, col_data in enumerate(row_data):
        ws.cell(row=ri+1,column=ci+1,value=col_data)
wb.save(fname)